### GASS evolution

nb to test the evolution functions 

In [16]:
using PyCall
using DataFrames
using Printf

rootdir = "/home/stephane/Science/ALMA/ArrayConfig/GASS"

push!(LOAD_PATH,"$rootdir/master/src")
using GASS
using Printf

import PyPlot
import Random

@pyimport astropy.coordinates as coord
@pyimport astropy.modeling.models as am
@pyimport astropy.modeling.fitting as mf

@pyimport numpy as np


## directory
datadir = "$rootdir/master/data"
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/test"

cd(wdir)

In [17]:
mutable struct _population
    age::Int32                 ## age
    subarr::Array{Array{Int,1},2}     ## {Population,Subindices}
    fitness::Array{Float64,2}  ## fitness of each subarray
    score::Array{Float64,1}    ## global score of each set
    # prop::Array{Array{Float64,2},2}     ## beam and mrs for each subarray (to be confirmed)    
end

In [18]:
## fitness function for a subarray
## cfg: GASS parameters
## subarrid: subarray id (int)
## subind: subarray indices in the main Arr.
##
function _fitness_subarray(cfg, subarrid, subind)
    subarr = cfg.arr[subind,:]
    
    bl= calc_baselines(subarr)
    uv= calc_uv(bl, cfg.obs.Source_Hour_Angle ,  cfg.obs.Source_Declination)
    h , dr=  calc_dirtybeam(uv , 255, 127, robust=0.5)
    b= fit_beam(h , dr)
    mrs= calc_mrs(uv)
    
    #@printf("## subarray fitness \n")
    #@printf("## beam:")
    #println(b)
    #@printf("## MRS: %3.3f \n", mrs)
    
    res= 0
    res += cfg.wei.Weight_Spatial_Resolution[subarrid]*abs(b.ar-cfg.sub.Spatial_Resolution[subarrid])
    res += cfg.wei.Weight_Elongation[subarrid]*abs(b.e-cfg.sub.Elongation[subarrid])*sign(b.e-
        cfg.sub.Elongation[subarrid])
    res += cfg.wei.Weight_Sidelobe_Levels[subarrid]*abs(b.sidelobe-
        cfg.sub.Sidelobe_Level[subarrid])*sign(b.sidelobe-cfg.sub.Sidelobe_Level[subarrid])
    res += cfg.wei.Weight_Maximum_Recoverable_Scale[subarrid]*abs(mrs-
        cfg.sub.Maximum_Recoverable_Scale[subarrid])*sign(cfg.sub.Maximum_Recoverable_Scale[subarrid]-mrs)
    
    return(res)
end

_fitness_subarray (generic function with 1 method)

In [19]:
## Creation pf a population
## 

function _create_population(cfg)
    
    pop= Array{Array{Int,1},2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    fitness= Array{Float64,2}(undef,cfg.ga.Population_Size,  cfg.obs.Subarray_Number)
    score=  Array{Float64,1}(undef,cfg.ga.Population_Size)
    
    subind= collect(1:cfg.obs.Antenna_Number)
    
    for i in 1:cfg.ga.Population_Size
        Random.shuffle!(subind)
        for j in 1:cfg.obs.Subarray_Number
            pop[i,j]= subind[cfg.sub.Subrange[j]]
            fitness[i,j]= _fitness_subarray(cfg, j, pop[i,j])
            println(i," ",j," ",fitness[i,j])
        end
        score[i]= -sum(cfg.wei.Weight_Subarray[:] .* fitness[i,:])
        println(score[i])
    end
    
    _pop= _population(0, pop, fitness, score)
    
    return(_pop)
end

_create_population (generic function with 1 method)

In [ ]:
## get best parents
##

function get_elitism()

In [20]:
macro main(inpfile)
    cfg = read_cfg(inpfile , verbose=true)
    
    println("##")
    println("## Create population...")
    p0= _create_population(cfg)
    
    ## PLOT ####
    #PyPlot.imshow(h)
    #PyPlot.colorbar()
    #PyPlot.show()
    
end

@main("../master/data/GA_Inputs_O-3.txt.julia")

## Input Parameters for GASS 
### Configuration file: ../master/data/O-3.cfg 
### Obs. Latitude: -23.026 
### Source Declination: -50.0 
### HA: 0.0 
### Antenna number: 50 
### Subarray number: 4 
##
## Subarray Parameters
### Pads per subarray: [30, 10, 7, 3]
### Name: ["Sub_1", " Sub_2 ", " Sub_3 ", " Sub_4"]
### AR: [2.3, 3.0, 3.5, 3.5]
### MRS: [20.0, 20.0, 10.0, 10.0]
### elongation: [1.2, 1.3, 2.0, 3.0]
### sidelobe: [10.0, 20.0, 60.0, 60.0]
##
## GA parameters
### Iterations: 100 
### Population size: 10 
### Mutation rate: 0.050 
### Tournament size: 5 
### Elitism: 5 
##
## Weights
### Subarray weights: [0.25, 0.25, 0.25, 0.25]
### AR weights: [0.25, 0.25, 0.25, 0.25]
### MRS weights: [0.25, 0.25, 0.25, 0.25]
### elongation weights: [0.25, 0.25, 0.25, 0.25]
### sidelobe weights: [0.25, 0.25, 0.25, 0.25]
##
##
## Create population...
1 1 -2.8051851773080823
1 2 0.7039539226121061
1 3 -15.658596634649843
1 4 -11.370570151187483
7.282599510133325
2 1 2.4716972809935527
2 2 -1.47